In [1]:
import pandas as pd
import os
import configparser
from neo4j import GraphDatabase
import textwrap

In [2]:
# Using an ini file for credentials, otherwise providing defaults
HOST = 'neo4j://localhost'
DATABASE = 'neo4j'
PASSWORD = 'password'

NEO4J_CONF_FILE = 'neo4j.ini'

if NEO4J_CONF_FILE is not None and os.path.exists(NEO4J_CONF_FILE):
    config = configparser.RawConfigParser()
    config.read(NEO4J_CONF_FILE)
    HOST = config['NEO4J']['HOST']
    DATABASE = config['NEO4J']['DATABASE']
    PASSWORD = config['NEO4J']['PASSWORD']
    print('Using custom database properties')
else:
    print('Could not find database properties file, using defaults')

Using custom database properties


In [4]:
# create driver
driver = GraphDatabase.driver(HOST, auth=(DATABASE, PASSWORD))

In [5]:
# helper function
def run(driver, query, params=None):
    with driver.session() as session:
        if params is not None:
            return [r for r in session.run(query, params)]
        else:
            return [r for r in session.run(query)]

# Query

## Find relevant information about a POI

In [7]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi {name: "Tiger Brewery Tour"})
    MATCH (poi)-[:BELONGS_TO]->(category:Category)
    MATCH (poi)-[:LOCATED_AT]->(region:Region)
    RETURN  poi.id AS PoiId, 
            poi.name AS PoiName, 
            category.name AS Category,
            region.name AS Region,
            poi.description AS PoiDescription, 
            poi.address AS PoiAddress, 
            poi.avgRating AS PoiRating, 
            poi.duration AS PoiDuration, 
            poi.openingHours AS PoiOpeningHours, 
            poi.price AS PoiPrice,
            poi.numReviews AS PoiNumReviews,
            poi.numReviews_1 AS PoiNumRating1,
            poi.numReviews_2 AS PoiNumRating2,
            poi.numReviews_3 AS PoiNumRating3,
            poi.numReviews_4 AS PoiNumRating4,
            poi.numReviews_5 AS PoiNumRating5,
            poi.url AS TripAdvisorUrl
    """)
)

[<Record PoiId=1815807 PoiName='Tiger Brewery Tour' Category='Breweries' Region='Tuas' PoiDescription='Step into the home of Tiger Beer and see how we brew Singapore’s iconic beer, with just four high-quality ingredients! The Tiger Brewery Tour takes you behind each step of the brewing process – from milling the malt to packaging. To round it off, you will try your hand at tapping the perfect pint of draught beer, and sample some of the freshest beers that will touch your lips. Book your tour today!' PoiAddress='459 Jalan Ahmad Ibrahim, Singapore 639934 Singapore' PoiRating=4.0 PoiDuration='2-3 hours' PoiOpeningHours='1:00 PM - 5:00 PM' PoiPrice=21.33 PoiNumReviews=270 PoiNumRating1=10 PoiNumRating2=17 PoiNumRating3=48 PoiNumRating4=94 PoiNumRating5=101 TripAdvisorUrl='https://www.tripadvisor.com/Attraction_Review-g294265-d1815807-Reviews-Tiger_Brewery_Tour-Singapore.html'>]

## Find all information about a user

In [7]:
run(driver, textwrap.dedent("""\
    MATCH (user:User {id: 1})-[:FROM]-(origin:Origin)
    RETURN  user.id AS UserId, 
            user.name AS UserName, 
            origin.name AS Origin
    """)
)

[<Record UserId=1 UserName='kra63' Origin='Sydney, Australia'>]

## Find point of interest that offer "Gardens" experiences.

In [23]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)-[:BELONGS_TO]->(category:Category {name: "Gardens"})
    RETURN poi.name AS PoiName, category.name AS CategoryName
    """)
)

[<Record PoiName='Gallop Extension' CategoryName='Gardens'>,
 <Record PoiName='Floral Fantasy' CategoryName='Gardens'>,
 <Record PoiName='Gardens By The Bay East' CategoryName='Gardens'>,
 <Record PoiName='Singapore airport' CategoryName='Gardens'>,
 <Record PoiName='Cloud Forest' CategoryName='Gardens'>,
 <Record PoiName='Singapore Botanic Gardens' CategoryName='Gardens'>,
 <Record PoiName='Gardens by the Bay' CategoryName='Gardens'>]

## List all point of interests in Region “Outram” along with their address.

In [24]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)-[:LOCATED_AT]->(region:Region {name: "Outram"})
    RETURN poi.name AS PoiName, region.name AS RegionName, poi.address AS PoiAddress
    """)
)

[<Record PoiName='Duxton Plain Park' RegionName='Outram' PoiAddress='New Bridge Road, Singapore 089140 Singapore'>,
 <Record PoiName='Virtual Room: Virtual Reality Singapore' RegionName='Outram' PoiAddress='211 New Bridge Road Floor 3 Lucky Chinatown, Singapore 059432 Singapore'>,
 <Record PoiName='Tea Chapter' RegionName='Outram' PoiAddress='9 Neil Road, Singapore 088808 Singapore'>,
 <Record PoiName='Zenith Wholesale Enterprise' RegionName='Outram' PoiAddress='50 Chin Swee Road Thong Chai Building, Singapore 169874 Singapore'>,
 <Record PoiName='Telok Ayer Street' RegionName='Outram' PoiAddress=''>,
 <Record PoiName='Tanjong Pagar' RegionName='Outram' PoiAddress=''>,
 <Record PoiName='Chinatown' RegionName='Outram' PoiAddress='Crot Trengganu, Singapore 048942 Singapore'>,
 <Record PoiName='VR World Singapore' RegionName='Outram' PoiAddress='8A Sago Street, Singapore 059012 Singapore'>,
 <Record PoiName='Chinese New Year (China Town)' RegionName='Outram' PoiAddress='Crot | Trengganu C

## List the reference price of a specific point of interest, namely “Singapore Zoo”.

In [25]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi {name: "Singapore Zoo"})
    RETURN poi.name AS PoiName, poi.price AS PoiPrice
    """)
)

[<Record PoiName='Singapore Zoo' PoiPrice=33.49>]

## List point of interest with pricing information within a price range from SGD 10 to SGD 20.

In [9]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)
    WHERE poi.price >= 10 AND poi.price <= 20
    RETURN poi.name AS PoiName, poi.price AS PoiPrice
    """)
)

[<Record PoiName='Cloud Forest' PoiPrice=11.42>,
 <Record PoiName='Asian Civilisations Museum' PoiPrice=12.01>,
 <Record PoiName='Changi Experience Studio' PoiPrice=17.09>,
 <Record PoiName='Floral Fantasy' PoiPrice=11.42>,
 <Record PoiName='SkyHelix Sentosa' PoiPrice=13.7>]

## List historical museums that offer insights into Singapore's history for History Enthusiasts.

In [27]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)-[:BELONGS_TO]->(category:Category {name: "History Museums"})
    RETURN poi.name AS PoiName, category.name AS CategoryName
    """)
)

[<Record PoiName='Former Ford Factory' CategoryName='History Museums'>,
 <Record PoiName='Asian Civilisations Museum' CategoryName='History Museums'>,
 <Record PoiName='Changi Chapel and Museum' CategoryName='History Museums'>]

## List point of interests with scenic landscapes, or iconic landmarks for photography enthusiasts and sightseeing.

In [28]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)-[:BELONGS_TO]->(category:Category)
    WHERE category.name IN ["Points of Interest & Landmarks", "Lookouts", "Observation Decks & Towers", "Observatories & Planetariums", "Scenic Walking Areas"]
    RETURN DISTINCT poi.name AS PoiName, category.name AS CategoryName
    """)
)

[<Record PoiName='Telok Ayer Street' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='SkyHelix Sentosa' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Supertree Grove' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Sands Skypark Observation Deck' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Singapore Flyer' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Cloud Forest' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Gardens by the Bay' CategoryName='Points of Interest & Landmarks'>,
 <Record PoiName='Sands Skypark Observation Deck' CategoryName='Observation Decks & Towers'>,
 <Record PoiName='Singapore Flyer' CategoryName='Observation Decks & Towers'>,
 <Record PoiName='Supertree Grove' CategoryName='Observatories & Planetariums'>,
 <Record PoiName='Faber Peak' CategoryName='Scenic Walking Areas'>,
 <Record PoiName='Singapore Cable Car' CategoryName='Scenic Walking Areas'>,
 <

## List point of interests that only takes 1 to 2 hours to visit.

In [31]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi {duration:"1-2 hours"})
    RETURN poi.name AS PoiName, poi.duration AS PoiDuration
    """)
)

[<Record PoiName='Singapore Botanic Gardens' PoiDuration='1-2 hours'>,
 <Record PoiName='Singapore Flyer' PoiDuration='1-2 hours'>,
 <Record PoiName='Sands Skypark Observation Deck' PoiDuration='1-2 hours'>,
 <Record PoiName='Supertree Grove' PoiDuration='1-2 hours'>,
 <Record PoiName='Chinatown' PoiDuration='1-2 hours'>,
 <Record PoiName='Merlion Park' PoiDuration='1-2 hours'>,
 <Record PoiName='Singapore Cable Car' PoiDuration='1-2 hours'>,
 <Record PoiName='Changi Chapel and Museum' PoiDuration='1-2 hours'>,
 <Record PoiName='Changi Experience Studio' PoiDuration='1-2 hours'>,
 <Record PoiName='Virtual Room: Virtual Reality Singapore' PoiDuration='1-2 hours'>,
 <Record PoiName='Floral Fantasy' PoiDuration='1-2 hours'>,
 <Record PoiName='Armenian Church' PoiDuration='1-2 hours'>,
 <Record PoiName='Tea Chapter' PoiDuration='1-2 hours'>,
 <Record PoiName='VR World Singapore' PoiDuration='1-2 hours'>]

## List point of interest that has an average rating of 4.

In [33]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi {avgRating:4})
    RETURN poi.name AS PoiName, poi.avgRating AS PoiAvgRating
    """)
)

[<Record PoiName='Chinatown' PoiAvgRating=4.0>,
 <Record PoiName='Merlion Park' PoiAvgRating=4.0>,
 <Record PoiName='Maxwell Food Centre' PoiAvgRating=4.0>,
 <Record PoiName='Tiger Brewery Tour' PoiAvgRating=4.0>,
 <Record PoiName='Thian Hock Keng Temple' PoiAvgRating=4.0>,
 <Record PoiName='Tekka Centre' PoiAvgRating=4.0>,
 <Record PoiName='Pasir Ris Park' PoiAvgRating=4.0>,
 <Record PoiName='Ngee Ann City' PoiAvgRating=4.0>,
 <Record PoiName='City Square Mall' PoiAvgRating=4.0>,
 <Record PoiName='Changi Beach' PoiAvgRating=4.0>,
 <Record PoiName='313@somerset' PoiAvgRating=4.0>,
 <Record PoiName='Floral Fantasy' PoiAvgRating=4.0>,
 <Record PoiName='Sri Krishnan Temple' PoiAvgRating=4.0>,
 <Record PoiName='Joo Chiat/Katong' PoiAvgRating=4.0>,
 <Record PoiName='Lazarus Island' PoiAvgRating=4.0>,
 <Record PoiName='Armenian Church' PoiAvgRating=4.0>,
 <Record PoiName='Central Fire Station' PoiAvgRating=4.0>,
 <Record PoiName='Hill Street Building' PoiAvgRating=4.0>,
 <Record PoiName='Old

## List all the point of interest that are in the same region as a specific point of interest, namely “Singapore Botanic Gardens”.

In [34]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi {name: "Singapore Botanic Gardens"})-[:LOCATED_AT]->(region:Region)
    WITH region
    MATCH (poi:Poi)-[:LOCATED_AT]->(region:Region)
    RETURN poi.name AS PoiName, region.name AS RegionName
    """)
)

[<Record PoiName='Gallop Extension' RegionName='Tanglin'>,
 <Record PoiName='Singapore Botanic Gardens' RegionName='Tanglin'>]

## Find Users who Wrote Reviews with a Rating of 5

In [10]:
run(driver, textwrap.dedent("""\
    MATCH (user:User)-[:WROTE]->(review:Review {rating: 5})-[:RATED ]->(poi:Poi)
    RETURN user.name AS User, review.title AS ReviewTitle, poi.name AS PoiName
    """)
)

[<Record User='Nigel3011' ReviewTitle='Chihuly Glass Floral Exhibition' PoiName='Gardens by the Bay'>,
 <Record User='178Janie' ReviewTitle='Gardens by the bay' PoiName='Gardens by the Bay'>,
 <Record User='StephenV397' ReviewTitle='Still new' PoiName='Gardens by the Bay'>,
 <Record User='WilliamO978' ReviewTitle='Absolutely amazing!' PoiName='Gardens by the Bay'>,
 <Record User='WarwickHall101' ReviewTitle='Beat the heat' PoiName='Gardens by the Bay'>,
 <Record User='PNC85' ReviewTitle='Utopian city of the future' PoiName='Gardens by the Bay'>,
 <Record User='cburni' ReviewTitle='Gardens with a difference' PoiName='Gardens by the Bay'>,
 <Record User='TravellingSandy4' ReviewTitle='A Pleasant Way to Spend Some Hours' PoiName='Gardens by the Bay'>,
 <Record User='SydneyTraveloz' ReviewTitle='Looking into the future' PoiName='Gardens by the Bay'>,
 <Record User='narellew186' ReviewTitle='oasis' PoiName='Gardens by the Bay'>,
 <Record User='QCV' ReviewTitle='Went twice' PoiName='Gardens 

## Find the Top-Rated POIs

In [6]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)<-[r:RATED]-(review:Review)
    WITH poi, AVG(review.rating) AS avgRating
    ORDER BY avgRating DESC
    RETURN poi.name AS PoiName, avgRating AS AverageRating
    LIMIT 10
    """)
)

[<Record PoiName='Gardens by the Bay' AverageRating=5.0>,
 <Record PoiName='Commune Kitchen' AverageRating=5.0>,
 <Record PoiName='Flyer Lounge @ Singapore Flyer' AverageRating=5.0>,
 <Record PoiName='Low Salt Low Sugar' AverageRating=5.0>,
 <Record PoiName='Singapore Botanic Gardens' AverageRating=5.0>,
 <Record PoiName='Singapore Zoo' AverageRating=5.0>,
 <Record PoiName='Cathedral of the Good Shepherd' AverageRating=5.0>,
 <Record PoiName='Floral Fantasy' AverageRating=5.0>,
 <Record PoiName='SkyHelix Sentosa' AverageRating=5.0>,
 <Record PoiName='Lazarus Island' AverageRating=5.0>,
 <Record PoiName='Central Fire Station' AverageRating=5.0>,
 <Record PoiName='CÉ LA VI Singapore' AverageRating=5.0>,
 <Record PoiName='Tea Chapter' AverageRating=5.0>,
 <Record PoiName='Old Supreme Court Building' AverageRating=5.0>,
 <Record PoiName='Geylang Serai' AverageRating=5.0>,
 <Record PoiName='Merlion Park' AverageRating=5.0>,
 <Record PoiName='Maxwell Food Centre' AverageRating=5.0>,
 <Record

## Find Users who Reviewed a Specific POI

In [8]:
run(driver, textwrap.dedent("""\
    MATCH (user:User)-[:WROTE]->(review:Review)-[:RATED]->(poi:Poi {name: 'Pasir Ris Park'})
    RETURN user.name AS User, review.title AS ReviewTitle, review.rating AS Rating
    """)
)

[<Record User='mostlyharmless' ReviewTitle='Pasir Ris park, revisited' Rating=3.0>,
 <Record User='SaintStacey' ReviewTitle='Big Playground' Rating=3.0>,
 <Record User='EarthlingOnline' ReviewTitle='Revisited: “Beachfront” with mangrove swamp, bikes for hire, and more.  But..' Rating=5.0>,
 <Record User='KGB777' ReviewTitle='Return visit' Rating=4.0>,
 <Record User='dynamic14' ReviewTitle='A great place to unwind' Rating=3.0>,
 <Record User='HeatSeekerWrexham_UK' ReviewTitle='Wildlife reserve easy to reach by MRT' Rating=5.0>,
 <Record User='Jacky84' ReviewTitle='Very small park' Rating=5.0>,
 <Record User='VishalSawant' ReviewTitle='Natures Trail' Rating=3.0>,
 <Record User='779waimunl' ReviewTitle='Best park ever at Area 3' Rating=5.0>,
 <Record User='Matthew77ask' ReviewTitle='Nice place to capture nature' Rating=5.0>,
 <Record User='AngyChin' ReviewTitle='Clean, quiet park serenely curving by the sea' Rating=3.0>,
 <Record User='mostlyharmless' ReviewTitle='Lovely trees.. but dista

## Find the Top 10 Active Users with Most Reviews

In [5]:
run(driver, textwrap.dedent("""\
    MATCH (user:User)-[:WROTE]->(review:Review)
    WITH user, COUNT(*) AS reviewCount
    ORDER BY reviewCount DESC
    RETURN user.name AS UserName, user.id AS UserId, reviewCount AS ReviewCount
    """)
)

[<Record UserName='KGB777' UserId=3098 ReviewCount=34>,
 <Record UserName='bcheong' UserId=21691 ReviewCount=19>,
 <Record UserName='Travelove58' UserId=769 ReviewCount=18>,
 <Record UserName='drchlow' UserId=1253 ReviewCount=18>,
 <Record UserName='alhasan1909' UserId=6967 ReviewCount=18>,
 <Record UserName='619jeffry' UserId=24816 ReviewCount=14>,
 <Record UserName='Oldjack' UserId=16814 ReviewCount=13>,
 <Record UserName='Ogi0' UserId=22145 ReviewCount=13>,
 <Record UserName='41gavinc' UserId=5698 ReviewCount=12>,
 <Record UserName='N8699PYpeterw' UserId=22779 ReviewCount=12>,
 <Record UserName='Aryan2007' UserId=22902 ReviewCount=11>,
 <Record UserName='Kim025Singapore' UserId=142 ReviewCount=11>,
 <Record UserName='S9788OVmohameds' UserId=28005 ReviewCount=11>,
 <Record UserName='blairkk25' UserId=21500 ReviewCount=11>,
 <Record UserName='0_Gyrfalcon_0' UserId=25522 ReviewCount=10>,
 <Record UserName='vipi0017' UserId=13624 ReviewCount=10>,
 <Record UserName='7SE3' UserId=29231 Re

## Find the Number of Reviews per Rating

In [11]:
run(driver, textwrap.dedent("""\
    MATCH (review:Review)
    RETURN review.rating AS Rating, COUNT(*) AS ReviewCount
    ORDER BY Rating
    """)
)

[<Record Rating=2.0 ReviewCount=7>,
 <Record Rating=3.0 ReviewCount=4676>,
 <Record Rating=4.0 ReviewCount=1225>,
 <Record Rating=5.0 ReviewCount=79721>]

## Find Users who Reviewed POIs in a Specific Region

In [13]:
run(driver, textwrap.dedent("""\
    MATCH (user:User)-[:WROTE]->(review:Review)-[:RATED]->(poi:Poi)-[:LOCATED_AT]->(region:Region {name: 'Marina Centre'})
    RETURN user.name AS User, poi.name AS PoiName, region.name AS Region
    """)
)

[<Record User='davidhF6017TS' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='ausmike_1968' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='AlifromCante' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='a_wayof_travel' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='Guido1008' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='kgprashanth' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='723teresaw' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='decenu' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='CharnP_13' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='KAweilWe' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='Miffed1' PoiName='Waterfront Promenade' Region='Marina Centre'>,
 <Record User='patricklR6608NR' PoiName='Waterfront Promenade' Region='Marina Centre'>

## Find POIs with the Highest Number of Reviews

In [35]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)<-[rated:RATED]-(:Review)
    WITH poi, COUNT(rated) AS numReviews
    ORDER BY numReviews DESC
    RETURN poi.name AS PoiName, numReviews AS NumReviews
    """)
)

[<Record PoiName='Gardens by the Bay' NumReviews=21381>,
 <Record PoiName='Singapore Mass Rapid Transit  (SMRT)' NumReviews=18064>,
 <Record PoiName='Singapore Zoo' NumReviews=7453>,
 <Record PoiName='Singapore Botanic Gardens' NumReviews=6887>,
 <Record PoiName='Singapore Flyer' NumReviews=5324>,
 <Record PoiName='Cloud Forest' NumReviews=5126>,
 <Record PoiName='Sands Skypark Observation Deck' NumReviews=4707>,
 <Record PoiName='Chinatown' NumReviews=4319>,
 <Record PoiName='Merlion Park' NumReviews=2199>,
 <Record PoiName='Supertree Grove' NumReviews=2047>,
 <Record PoiName='Singapore Cable Car' NumReviews=1329>,
 <Record PoiName='Waterfront Promenade' NumReviews=1319>,
 <Record PoiName='CÉ LA VI Singapore' NumReviews=902>,
 <Record PoiName='Changi Chapel and Museum' NumReviews=893>,
 <Record PoiName='Asian Civilisations Museum' NumReviews=773>,
 <Record PoiName='The Shoppes at Marina Bay Sands' NumReviews=662>,
 <Record PoiName='Maxwell Food Centre' NumReviews=247>,
 <Record PoiNam

## Find Users who Visited POIs in a Specific Category "Theater & Performances"

In [36]:
run(driver, textwrap.dedent("""\
    MATCH (user:User)-[:WROTE]->(review:Review)-[:RATED]->(poi:Poi)-[:BELONGS_TO]->(category:Category {name: 'Theater & Performances'})
    RETURN user.name AS UserName, poi.name AS PoiName, category.name AS CategoryName
    """)
)

[<Record UserName='pierreny' PoiName='Moshi Moshi Bollywood' CategoryName='Theater & Performances'>,
 <Record UserName='georgecO1950TW' PoiName='Moshi Moshi Bollywood' CategoryName='Theater & Performances'>,
 <Record UserName='div106' PoiName='Moshi Moshi Bollywood' CategoryName='Theater & Performances'>,
 <Record UserName='munstar' PoiName='Moshi Moshi Bollywood' CategoryName='Theater & Performances'>,
 <Record UserName='pt2pt3' PoiName='Moshi Moshi Bollywood' CategoryName='Theater & Performances'>]

## Find the Average Rating for POIs in Each Region

In [37]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)-[:LOCATED_AT]->(region:Region)
    WITH region, AVG(poi.avgRating) AS avgRating
    RETURN region.name AS Region, avgRating AS AverageRating
    """)
)

[<Record Region='Central Area/City Area' AverageRating=4.333333333333333>,
 <Record Region='Tanglin' AverageRating=4.5>,
 <Record Region='Central Water Catchment' AverageRating=4.5>,
 <Record Region='Marina Centre' AverageRating=4.333333333333333>,
 <Record Region='Marina Bay' AverageRating=4.4>,
 <Record Region='Outram' AverageRating=4.136363636363636>,
 <Record Region='Mount Faber Park' AverageRating=4.5>,
 <Record Region='Changi West' AverageRating=4.5>,
 <Record Region='City Hall' AverageRating=4.125>,
 <Record Region='Central Business District' AverageRating=2.0>,
 <Record Region='Tuas' AverageRating=4.0>,
 <Record Region='Changi' AverageRating=4.5>,
 <Record Region='Little India' AverageRating=4.0>,
 <Record Region='Toh Guan' AverageRating=3.5>,
 <Record Region='Pasir Ris' AverageRating=4.0>,
 <Record Region='Boulevard' AverageRating=4.0>,
 <Record Region='Lavender' AverageRating=4.0>,
 <Record Region='Somerset' AverageRating=4.0>,
 <Record Region='Bukit Batok' AverageRating=4.5>

## Find POIs with the Highest Number of 5-Star Ratings

In [38]:
run(driver, textwrap.dedent("""\
    MATCH (poi:Poi)<-[rated:RATED]-(:Review {rating: 5})
    WITH poi, COUNT(rated) AS numFiveStarRatings
    ORDER BY numFiveStarRatings DESC
    RETURN poi.name AS PoiName, numFiveStarRatings AS NumFiveStarRatings
    """)
)

[<Record PoiName='Gardens by the Bay' NumFiveStarRatings=21381>,
 <Record PoiName='Singapore Mass Rapid Transit  (SMRT)' NumFiveStarRatings=18064>,
 <Record PoiName='Singapore Zoo' NumFiveStarRatings=7453>,
 <Record PoiName='Singapore Botanic Gardens' NumFiveStarRatings=6887>,
 <Record PoiName='Singapore Flyer' NumFiveStarRatings=5324>,
 <Record PoiName='Cloud Forest' NumFiveStarRatings=5126>,
 <Record PoiName='Sands Skypark Observation Deck' NumFiveStarRatings=4707>,
 <Record PoiName='Merlion Park' NumFiveStarRatings=2199>,
 <Record PoiName='Supertree Grove' NumFiveStarRatings=2047>,
 <Record PoiName='Singapore Cable Car' NumFiveStarRatings=1329>,
 <Record PoiName='Waterfront Promenade' NumFiveStarRatings=1319>,
 <Record PoiName='CÉ LA VI Singapore' NumFiveStarRatings=902>,
 <Record PoiName='Changi Chapel and Museum' NumFiveStarRatings=893>,
 <Record PoiName='Asian Civilisations Museum' NumFiveStarRatings=773>,
 <Record PoiName='Maxwell Food Centre' NumFiveStarRatings=247>,
 <Record P

# Close the driver

In [ ]:
driver.close()